In [14]:
import xarray
from distributed import Client
import sys
sys.path.insert(1, '../heatwave_analysis_package/src')
import hdp
import cftime
import numpy as np

client = Client('localhost:8786')
client

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10>